# Visual Recognition Tooling

This is a set of tools that should help to get up to speed when delivering Visual Recognition projects. It provides helpers to simplify the training, testing and evaluation of classifiers.

## Features
- Binary classifier traning
- K-Fold Cross Validation
- Evaluation of binary and multi class classifiers
- Persisting of train, test and result sets

## Image Corpus Layout

Currently the tooling is working with image corpora that are file and folder based. An image corpus can consist of several folders. Each folder represents a class the respective classifier will be able to recognize. Each class folder  contains all images that will be used to train and test the classifier on this class. If only one class per classifier is given, a folder called negative_examples is needed as well.

To get a better understanding of the layout, take a look at this sample folder hierarchy:

```
 ./corpora_folder_cars
     /bmw_corpus
         /three
             320.jpg
             330.jpg
         /five
             530.jpg
             550.jpg
         /seven
             750.jpg
             760.jpg
     /audi_corpus
         /athree
             a3.jpg
         /afour
             a4.jpg
     /mercedes_corpus
         /sclass
             s500.jpg
         /negative_examples
             eclass.jpg
```

# Initialization

In [ ]:
# import basic libraries
import time
import os
import sys
import json
import pickle
import shutil
import configparser
import datetime
import numpy as np
import pandas as pd

# import sklearn helpers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

# import custom VR tooling libs
import vrtool

# Configuration


When using this tool for the first time, you'll find a file called **dummy.config.ini** which needs to be copied and renamed to **config.ini**.


Configure *your* tool by entering your IAM API key and URL of the Visual Recognition service instance.
```
[vr]
IAM_API_KEY:your_IAM_api_key
URL:your_service_url
```

# Corpus Overview & Statistics

The following section provides an extensive overview of the image corpus and statistics of the same.

In [ ]:
# The name of the folder that contains the corpora, currently relative to notebook location
corpora_folder_name = '../corpus'
config_name = 'config.ini'

runner = vrtool.Runner(corpora_folder_name, config_name=config_name)
corpora = runner.get_available_corpora()

# Print a summary of the available corpora in the corpora directory
print()
print('Available image corpora:')
print('\n'.join('{}: {[0]}'.format(*el) for el in enumerate(corpora)))

In [ ]:
# Print a detailed overview of the different classes and their distribution within each corpus
corpora_info = runner.get_corpora_info(corpora)

# Create Test / Training Sets

In this step the training and test sets for a specific classifier are created in the follwoing steps:
1. Determine the corpus to be used by setting the **corpus_to_train** variable to a corpus name in your corpora folder (e.g. bmw)
2. Set the number of splits **k** for K-Fold cross validation
3. Check if **(k-1) * number of images per class > 10**, otherwise the class won't be used for training

In [ ]:
# Select the name of the corpus for which a classifier will be trained
corpus_name = 'CORPUS_TO_TRAIN'

# Train / Test split
test_ratio = 0.25

# Select the right corpus based on the value of corpus_to_train and filter out classes 
# with less than 10 images for training
img_info = [el['image_info'] for el in corpora_info if el['corpus_name'] == corpus_name ][0]
img_info = img_info.groupby('class_name').filter(lambda x: len(x) >= 10)

print("Classifier to be trained:", corpus_name)
print("Classes to be trained:", img_info['class_name'].unique())
print(img_info.class_name.value_counts())

## Sub Sample Images

In [ ]:
train_data, split_test_data = train_test_split(img_info, test_size = test_ratio, stratify=img_info['class_name'])
print("Training Set:")
print("-------------")
print(train_data.class_name.value_counts())
print("------------------------------")
print("Test Set:")
print("---------")
print(split_test_data.class_name.value_counts())

## (Optional) Export Images for Watson Studio

In [ ]:
KEEP_SRC_IMG = True
TARGET_FOLDER = './studio_data'

def move_img(src_path, target_path, keep_src_img=True):
    
    os.makedirs(os.path.dirname(target_path), exist_ok=True)
        
    if keep_src_img:
        shutil.copyfile(src_path, target_path)
    else:
        shutil.move(src_path, target_path)
        
def dump_data_df(data, target_folder, keep_src_img):
    for __, row in data.iterrows():
        src_img_path = row['image']
        target_img_path = os.path.join(target_folder, row['class_name'], row['image_name'] )
        move_img(src_img_path, target_img_path, keep_src_img)


In [ ]:
dump_data_df(train_data, os.path.join(TARGET_FOLDER, corpus_name + "_train"), KEEP_SRC_IMG)
dump_data_df(split_test_data, os.path.join(TARGET_FOLDER,corpus_name + "_test"), KEEP_SRC_IMG)

# Save Dataframes

All training and testing configurations will be saved as pickle file in the **modelconfiguration** folder referencing the image data used. 
That allows to reuse the data for retraining, testing or further analysis. 

In [ ]:
train_data.to_pickle("modelconfigurations/" + corpus_name + "_train_" +time.strftime("%d-%m-%Y-%H-%M-%S")+ ".pkl")
split_test_data.to_pickle("modelconfigurations/" + corpus_name + "_test_" +time.strftime("%d-%m-%Y-%H-%M-%S")+ ".pkl")

# Train Classifier

Train the classifier based on the experiments defined in the previous steps. This might take a couple of minutes depending on the number of training images used.

Internally the method is creating batches of images which are then zipped and sent to the Visual Recognition API for training.

You can also use previously created experiment pickle files to create classifiers by setting the **USE_EXTERNAL_EXPERIMENT_DATA** to **True** and specify the path to the external experiments.

## Load external experiment data sets for training
By deafult this cell does nothing and uses the data set that was created in this notebook.

You can also use previously created experiment pickle files to test classifiers by setting the **USE_EXTERNAL_EXPERIMENT_DATA** to **True** and specify the path to the external experiments.

In [ ]:
# Default: False -> use experiments created in this notebook
#          True -> use external experiments created earlier
USE_EXTERNAL_EXPERIMENT_DATA = False

# If True, specifiy external experiment data path (path_to_experiment.pkl)
EXTERNAL_EXPERIMENT_PATH='modelconfigurations/DATA.pkl'

if USE_EXTERNAL_EXPERIMENT_DATA:
    with open(EXTERNAL_EXPERIMENT_PATH,'rb') as f:
        experiments = pickle.load(f)

## Start Training

In [ ]:
start_time = time.time()
results = runner.train_classifier_from_data_frame(corpus_name, train_data)

print("Finished Training after %s seconds." % (time.time() - start_time))

# Test Classifier 

Performs classifier testing by packaging the image data into several zip files and sending them to the Visual Recognition Service for scoring. 

Main steps:
1. Get the relevant classifier ids to be used for testing
2. Perform the tests


## Get classifier Ids
Loads all available classifiers and tries to find the ones that match your corpus_to_train and cross validation folds.

If more than one classifier per cross validation iteration is found, you will see warnings. 
You need to interactively select the classifiers for each iteration.

In [ ]:
possible_classifiers = runner.vr_service.get_classifier_ids_by_name(corpus_name)
classifier_id = possible_classifiers[0]['classifier_id']

if(len(possible_classifiers) == 1):
    print("Classifier ID to be used:", classifier_id)
else:
    print("Please select the classifier you want to use for testing")
    print(json.dumps(possible_classifiers, indent=2))
    print('\nfirst classifier_id: ', possible_classifiers[0]['classifier_id'])

In [ ]:
# Or select classifier_id manually
#classifier_id = 'MANUALLY_SELECTED_ID'

## Load external experiment data set for testing
By deafult this cell does nothing and uses the data set that was created in this notebook.

You can also use previously created experiment pickle files to test classifiers by setting the **USE_EXTERNAL_EXPERIMENT_DATA** to **True** and specify the path to the external experiments.

In [ ]:
# If False, use experiments created in previous steps in this notebook
USE_EXTERNAL_EXPERIMENT_DATA = False

# Otherwise, external experiment data (filename.pkl) will be used from the specified path
EXTERNAL_EXPERIMENT_PATH='modelconfigurations/FILE.pkl'

if USE_EXTERNAL_EXPERIMENT_DATA:
    with open(EXTERNAL_EXPERIMENT_PATH,'rb') as f:
        experiments = pickle.load(f)

## Perform Tests

Test the classifier based on the experiments defined in the previous steps. This might take a couple of minutes depending on the number of images used for testing.



In [ ]:
start_time = time.time()

split_test_results = runner.test_classifier_with_data_frame(classifier_id, split_test_data)

print("Finished Testing after %s seconds." % (time.time() - start_time))

In [ ]:
parsed_result = runner.vr_service.parse_img_results(split_test_results)

# Evaluation

In this section the classifier performance is analyzed based on the tests that were performed in the previous steps.
A confusion matrix is created to analyze the true & false / positives & negatives.

## Load external data set for evaluation
By deafult this cell does nothing and uses the data set that was created in this notebook.

You can also use previously created experiment pickle files to test classifiers by setting the **USE_EXTERNAL_RESULT_DATA** to **True** and specify the path to the external experiments.

In [ ]:
# If False, use result data from the current test run in this notebook
USE_EXTERNAL_RESULT_DATA = False

# Otherwise, external result data (filename.pkl) will be used from the specified path
EXTERNAL_RESULT_PATH='modelconfigurations/EVAL.pkl'

if USE_EXTERNAL_RESULT_DATA:
    with open(EXTERNAL_RESULT_PATH,'rb') as f:
        experiments = pickle.load(f)

In [ ]:
## If you having existing results you only want to analyze, uncomment the following line and comment the next paragraph
#evaluation = pd.read_pickle('your_pickle_result_file.pkl')

# match results against expected classification results
evaluation = runner.merge_predicted_and_target_labels(split_test_data, split_test_results)

# save evaluation results for further analysis and documentation
evaluation.to_pickle("modelconfigurations/"+corpus_name + "_result_" +time.strftime("%d-%m-%Y-%H-%M-%S")+ ".pkl")

## Plot confusioin matrix as table

In [ ]:
# extract actual and predicted values from evaluation
y_actual = evaluation['class_name']
y_actual = y_actual.replace("negative_examples", "None")
y_actual = y_actual.replace("negative_examples", "None")
y_actual = y_actual.replace("negatives", "None")
y_pred = evaluation['predicted_class_1']

# create confusion matrix as table
pd.crosstab(y_pred,y_actual)

In [ ]:
# Test Performance for different thresholds
# TODO ROC
evaluation = runner.merge_predicted_and_target_labels(split_test_data, split_test_results)

thresholds = [0.5,0.6,0.7,0.75,0.8,0.85]
pd.options.display.max_colwidth = 600

for threshold in thresholds: 
    ev = evaluation
    ev.loc[ev['predicted_score_1'] < threshold,'predicted_class_1'] = 'None'
    y_actual_temp = ev['class_name']
    y_actual_temp = y_actual_temp.replace("negative_examples", "None").replace("negative_example", "None").replace("negatives", "None")
    y_pred_temp = ev['predicted_class_1']
    
    print("F1 Score:", threshold, metrics.f1_score(y_actual_temp, y_pred_temp,average='micro'))
    print("Overall Accuracy:", threshold ,metrics.accuracy_score(y_actual_temp, y_pred_temp))

## Plot confusioin matrix as chart

In [ ]:
## plot confusion matrix with threshold
threshold = 0.8

ev = runner.merge_predicted_and_target_labels(split_test_data, split_test_results)
ev.loc[ev['predicted_score_1'] < threshold,'predicted_class_1'] = 'None'
y_actual_temp = ev['class_name']
y_actual_temp = y_actual_temp.replace("negative_examples", "None").replace("negative_example", "None").replace("negatives", "None")
y_pred_temp = ev['predicted_class_1']

confmatrix = runner.get_confusion_matrix(y_actual, y_pred)

runner.plot_confusion_matrix(confmatrix, y_actual, y_pred, normalize=True,
                      title='Normalized confusion matrix')

# Visualize False Positives & False Negatives

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

## SET THRESHOLD
threshold = 0.8

ev = runner.merge_predicted_and_target_labels(split_test_data, split_test_results)
ev.loc[ev['predicted_score_1'] < threshold,'predicted_class_1'] = 'None'

y_actual_temp = ev['class_name']
y_actual_temp = y_actual_temp.replace("negative_examples", "None").replace("negative_example", "None").replace("negatives", "None")
y_pred_temp = ev['predicted_class_1']

fpfn = ev[ y_actual_temp!= y_pred_temp ]

image_count = fpfn.shape[0]

fig = plt.figure(figsize=(40,30))

columns = 5
idx = 0

for i, row in fpfn.iterrows():
    image = mpimg.imread(row['image_x'])
    ax = fig.add_subplot(int(image_count / columns + 1), columns, idx + 1)
    ax.set_title("is: "+row['class_name']
                         +"\n pred: "
                         + row['predicted_class_1']
                         +" \n file: "
                         +row['image_x'].split('/')[-1]
                         +" \n score: "
                         +str(row['predicted_score_1']), fontsize=25)
    idx = idx +1
    ax.imshow(image, aspect='auto')

plt.show()

# Histogram Threshold Performance

In [ ]:
x = evaluation['predicted_score_1']

n, bins, patches = plt.hist(x, 20, normed=0, facecolor='green', alpha=0.9)

In [ ]:
runner.zip_helper.clean_up()